# Programming Assignment: Рекомендательные системы

## Описание задачи

Небольшой интернет-магазин попросил вас добавить ранжирование товаров в блок «Смотрели ранее» — в нем теперь надо показывать не последние просмотренные пользователем товары, а те товары из просмотренных, которые он наиболее вероятно купит. Качество вашего решения будет оцениваться по количеству покупок в сравнении с прошлым решением в ходе А/В теста, т.к. по доходу от продаж статзначимость будет достигаться дольше из-за разброса цен. Таким образом, ничего заранее не зная про корелляцию оффлайновых и онлайновых метрик качества, в начале проекта вы можете лишь постараться оптимизировать `recall@k` и `precision@k`.

Это задание посвящено построению простых бейзлайнов для этой задачи: ранжирование просмотренных товаров по частоте просмотров и по частоте покупок. Эти бейзлайны, с одной стороны, могут помочь вам грубо оценить возможный эффект от ранжирования товаров в блоке — например, чтобы вписать какие-то числа в коммерческое предложение заказчику, а с другой стороны, могут оказаться самым хорошим вариантом, если данных очень мало (недостаточно для обучения даже простых моделей).

## Данные

Вам дается две выборки с пользовательскими сессиями – `id`-шниками просмотренных и `id`-шниками купленных товаров. Одна выборка будет использоваться для обучения (оценки популярностей товаров), а другая - для теста.

В файлах записаны сессии по одной в каждой строке. Формат сессии: `id` просмотренных товаров через `,` затем идёт `;`, после чего следуют `id` купленных товаров (если такие имеются), разделённые запятой. Например, `1,2,3,4;` или `1,2,3,4;5,6`.

Гарантируется, что среди `id` купленных товаров все различные.

**Важно:**
* Сессии, в которых пользователь ничего не купил, исключаем из оценки качества.
* Если товар не встречался в обучающей выборке, его популярность равна 0.
* Рекомендуем разные товары. И их число должно быть не больше, чем количество различных просмотренных пользователем товаров.
* Рекомендаций всегда не больше, чем минимум из двух чисел: количество просмотренных пользователем товаров и `k` в `recall@k`/`precision@k`.


## Задание

In [1]:
import pandas as pd
import numpy as np
from collections import OrderedDict
from collections import Counter

In [2]:
def f7(seq):
    seen = set()
    seen_add = seen.add
    return [x for x in seq if not (x in seen or seen_add(x))]

In [3]:
def str2list(string):
    if len(string) == 0:
        return None
    try: 
        return np.array(string.split(','),dtype=np.uint32)
    except ValueError as e:
        print('string: ', string)

### Задание 1. 

На обучении постройте частоты появления `id` в просмотренных и в купленных (`id` может несколько раз появляться в просмотренных, все появления надо учитывать).

In [ ]:
lst = ['1,2,3,1,6,1;1',

'1,4,5;5',

'4,3,6;4',

'2,1,2,4,5;',

'4,1,1,2,6;1,4']
lst = map(lambda x: x.split(';'),lst)

In [ ]:
test=pd.DataFrame(lst)

In [ ]:
test=test.applymap(str2list)

In [ ]:
test.columns=['see','buy']

In [ ]:
test

In [ ]:
test_see = np.concatenate(test.see.values)
test_see_vc = pd.Series(test_see).value_counts()

In [ ]:
buy_dict_train = test_see_vc.to_dict()
set_dic = set(buy_dict_train)

In [ ]:
test_dropna = test.dropna()

len_test = test_dropna.shape[0]

In [ ]:
session=test_dropna.loc[1]

In [ ]:
Recall(test_dropna.iloc[3],buy_dict_train,5)

In [ ]:
see=f7(session.see)
buy = session.buy

TDSK = set(buy_dict_train.keys())

see = [s for s in see if s in TDSK]

sorted(see,key= lambda x: buy_dict_train[x],reverse=True)
# reccomend = see[:k]

# bough_rec = np.intersect1d(reccomend,buy)

# print bough_rec.shape[0]/float(buy.shape[0])

In [ ]:
precision_buy_test_1 = 0.0
precision_buy_test_5 = 0.0

recall_buy_test_1 = 0.0
recall_buy_test_5 = 0.0
for ind in test_dropna.index:
    sess = test_dropna.loc[ind]
    
    res_prec1 = Precision(sess,buy_dict_train,set_dic,1)
#     print "res_prec1={}".format(res_prec1)
#     if not res_prec1 == 0:
    precision_buy_test_1 += res_prec1
    precision_buy_test_5 += Precision(sess,buy_dict_train,set_dic,5)
    recall_buy_test_1 += Recall(sess,buy_dict_train,set_dic,1)
    print "res_prec5={}".format(Recall(sess,buy_dict_train,set_dic,5))
    recall_buy_test_5 += Recall(sess,buy_dict_train,set_dic,5)

In [ ]:
ans1=[recall_buy_test_1/len_test,precision_buy_test_1/len_test,\
recall_buy_test_5/len_test,precision_buy_test_5/len_test]
print ans1

In [6]:
data_train = pd.read_table("coursera_sessions_train.txt",sep=';',header=None,converters={0:str2list, 1:str2list})
data_test = pd.read_table("coursera_sessions_train.txt",sep=';',header=None,converters={0:str2list, 1:str2list})

data_train.columns=['see','buy']
data_test.columns=['see','buy']

In [7]:
data_train.head()

,see,buy
0,"[0, 1, 2, 3, 4, 5]",None
1,"[9, 10, 11, 9, 11, 12, 9, 11]",None
2,"[16, 17, 18, 19, 20, 21]",None
3,"[24, 25, 26, 27, 24]",None
4,"[34, 35, 36, 34, 37, 35, 36, 37, 38, 39, 38, 39]",None


In [8]:
data_train.buy.dropna().shape

(3608,)

In [9]:
train_see = np.concatenate(data_train.see.values)
train_see_vc = pd.Series(train_see).value_counts()
ts_index = train_see_vc.index.values

In [10]:
train_buy = np.concatenate(data_train.buy.dropna().values)

In [11]:
train_buy_vc=pd.Series(train_buy).value_counts()

In [12]:
train_see_dic = train_see_vc.to_dict()

In [13]:
buy_dict_train = OrderedDict()

for ind in data_train.index:
    raw = data_train.loc[ind]
            
    buy_l_values = raw.buy
    if buy_l_values is None:
        continue
    for val in buy_l_values:
        if val in buy_dict_train.keys():
            buy_dict_train[val] += 1
        else:
            buy_dict_train[val] = 1

### Задание 2. 

Реализуйте два алгоритма рекомендаций: сортировка просмотренных `id` по популярности (частота появления в просмотренных), сортировка просморенных `id` по покупаемости (частота появления в покупках). Если частота одинаковая, то сортировать надо по возрастанию момента просмотра (чем раньше появился в просмотренных, тем больше приоритет)

In [58]:
def Precision(session, dic_sort,dic_set , k):
    see=session.see
    
    buy = session.buy
    
    k = min(k,see.shape[0])
    see = f7(see)
    
    see = [s for s in see if s in dic_set]
    
    if len(see) == 0:
        return 0
    
    sorted(see,key= lambda x: train_see_dic[x],reverse=True)
    reccomend = see[:k]
    
    bough_rec = np.intersect1d(reccomend,buy)
    
    return bough_rec.shape[0]/float(k)

In [59]:
def Recall(session, buy,dic_set, k):
    see=session.see
    
    buy = session.buy
    k = min(k,see.shape[0])
    t = min(see.shape[0],buy.shape[0])
    see = f7(see)
    
    
    
    see = [s for s in see if s in dic_set]
    
    if len(see) == 0:
        return 0
    
    sorted(see,key= lambda x: train_see_dic[x],reverse=True)
    reccomend = see[:k]
    
    bough_rec = np.intersect1d(reccomend,buy)
    
    return bough_rec.shape[0]/float(t)

### Задание 3. 

Для данных алгоритмов выпишите через пробел `AverageRecall@1`, `AveragePrecision@1`, `AverageRecall@5`, `AveragePrecision@5` на обучающей и тестовых выборках, округляя до 2 знака после запятой. Это будут ваши ответы в этом задании. Посмотрите, как они соотносятся друг с другом. Где качество получилось выше? Значимо ли это различие? Обратите внимание на различие качества на обучающей и тестовой выборке в случае рекомендаций по частотам покупки.

In [37]:
train_see_dic=buy_dict_train

In [38]:
test_dropna = data_test.dropna()

len_test = test_dropna.shape[0]

In [39]:
dict_set = set(train_see_dic.keys())

In [60]:
precision_buy_test_1 = 0.0
precision_buy_test_5 = 0.0

recall_buy_test_1 = 0.0
recall_buy_test_5 = 0.0
for ind in test_dropna.index:
    sess = test_dropna.loc[ind]

    precision_buy_test_1 += Precision(sess,train_see_dic,dict_set,1)
    precision_buy_test_5 += Precision(sess,train_see_dic,dict_set,5)
    recall_buy_test_1 += Recall(sess,train_see_dic,dict_set,1)
    recall_buy_test_5 += Recall(sess,train_see_dic,dict_set,5)
print recall_buy_test_1/len_test,precision_buy_test_1/len_test,\
recall_buy_test_5/len_test,precision_buy_test_5/len_test

0.701883950768 0.82012195122 0.930125307065 0.459160199557


In [ ]:
[0.4706289814521518, 0.5454545454545454, 0.8162751297729124, 0.2093680709534424]

In [54]:
ans1=[recall_buy_test_1/len_test,precision_buy_test_1/len_test,\
recall_buy_test_5/len_test,precision_buy_test_5/len_test]
print ans1

[0.7026453759962634, 0.8201219512195121, 0.9313157409540445, 0.49094142645972544]


In [55]:
ans1=map(lambda x : round(x,2),ans1)

In [56]:
str(ans1)

'[0.7, 0.82, 0.93, 0.49]'

In [57]:
with open("ans1","w") as f:
    for an in ans1:
        f.write(str(an)+' ')

### Дополнительные вопросы

* Обратите внимание, что при сортировке по покупаемости возникает много товаров с одинаковым рангом - это означает, что значение метрик будет зависеть от того, как мы будем сортировать товары с одинаковым рангом. Попробуйте убедиться, что при изменении сортировки таких товаров `recall@k` меняется. Подумайте, как оценить минимальное и максимальное значение `recall@k` в зависимости от правила сортировки.
* Мы обучаемся и тестируемся на полных сессиях (в которых есть все просмотренные за сессию товары). Подумайте, почему полученная нами оценка качества рекомендаций в этом случае несколько завышена.